<h1><font color=blue>Beginning with Constraint Programming</font></h1>
<H2><font color=green>The Golomb Ruler example</font></h2>

<h3>Problem description</h3>
<p>
A detailed description (from which this paragraph comes from) is available on <b>Wikipedia</b> at https://en.wikipedia.org/wiki/Golomb_ruler.
<p>
In mathematics, a Golomb ruler is a set of marks at integer positions along an imaginary ruler such that no two pairs of marks are the same distance apart. The number of marks on the ruler is its order, and the largest distance between two of its marks is its length. 
<p>
Following is an example of Golomb ruler of order 4 and length 6.
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Golomb_Ruler-4.svg/220px-Golomb_Ruler-4.svg.png"></center>
<p>
This problem is not only an intellectual problem. It has a lot of practical applications:
<ul>
<li> within Information Theory related to error correcting codes,
<li> the selection of radio frequencies to reduce the effects of intermodulation interference,
<li> the design of conference rooms, to maximize the number of possible configurations with a minimum of partitions:
</ul>
<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/Golomb_ruler_conference_room.svg/300px-Golomb_ruler_conference_room.svg.png"></center>


<h3>Modeling the problem</h3>
<p>
Constraint Programming is a programming paradigm that allows to express a problem using:
<ul>
<li> the unknowns of the problem (the <i>variables</i>),
<li> the constraints/laws/rules of the problem, mathematical expressions linking variables together (the <i>constraints</i>),
<li> what is to be optimized (the <i>objective function</i>).
</ul>
<p>
All this information, plus some configuration parameters, is aggregated into a single object called <i>model</i>. 
<p>
The remainder of this notebook describes in details how to build and solve this problem with IBM CP Optimizer, using its <i>DOcplex</i> Python modeling API.

<h3>Modeling with Python</h3>
<p>
Let's start to build our Golomb Ruler problem with IBM CP Optimizer for Python.
<p>
The first thing is to get access to the appropriate DOcplex library. The following code imports the appropriate package <i>docplex.cp</i>. If the import fails, an installation command is executed as it is probable that the package is not installed. 

In [ ]:
try:
    import docplex.cp
except:
    !pip install docplex

Note that the more global package <i>docplex</i> contains another subpackage <i>docplex.mp</i> that is dedicated to Mathematical Programming, another branch of optimization.

Now, we need to import all required modeling functions that are provided by the <i>docplex.cp</i> package:

In [ ]:
# Import Constraint Programming modelization functions
from docplex.cp.model import *

<h4>Define model input data</h4>
<p>
The first thing to define is the model input data.
In the case of the Golomb Ruler problem, there is only one input which is the order of the ruler, that is the number of marks:

In [ ]:
# Define required number of marks on the ruler
ORDER = 7

<h4>Create the model container</h4>
<p>
The model is represented by a Python object that is filled with the different model elements (variables, constraints, objective function, etc). The first thing to do is then to create such an object:

In [ ]:
# Create model object
mdl = CpoModel()

<h4>Define model variables</h4>
<p>
Now, we need to define the variables of the problem. As the expected problem result is the list of mark positions, the simplest choice is to create one integer variable to represent the position of each mark on the ruler.
<p>
Each variable has a a set of possible values called his <i>domain</i>. To reduce the search space, it is important to reduce this domain as far as possible.
<p>
In our case, we can naively estimate that the maximum distance between two adjacent marks is the order of the ruler minus one. Then the maximal position of a mark is (ORDER - 1)². Each variable domain is then limited to an interval [0..(ORDER - 1)²].
<p>
A list of integer variables can be defined using method <i>integer_var_list()</i>. In our case, defining one variable for each mark can be created as follows:

In [ ]:
# Create array of variables corresponding to rule marks
marks = integer_var_list(ORDER, 0, (ORDER - 1) ** 2, "M")

<h4>Define model constraints</h4>
<p>
We need to express that all possible distances between two marks must be different. To do this, we create an array that contains all these distances:

In [ ]:
# Create an array with all distances between all marks
dist = [marks[i] - marks[j] for i in range(1, ORDER) for j in range(0, i)]

We have used here the operator '-' to express the difference between variables. It may appear strange as the variables are not instanciated at that time, but the Python operator has been overloaded to construct a CP expression instead of attempting to compute the arithmetic difference. All other standard Python operators can be used to make operations between CP objects (<, >, <=, >=, ==, !=, +, -, /, *, &, |, //, **,  ...). Have a look to documentation for details.
<p>
To force all these distances to be different, we use the special <i>all_diff()</i> constraint as follows:

In [ ]:
# Force all distances to be different
mdl.add(all_diff(dist))

The call <i>mdl.add(...)</i> is necessary to express that the constraint must be added to the model.

<h4>Remove symmetries</h4>
<p>
The constraint we have expressed above is theoritically enough, and the model can be solved as it is.
<p>
However, it does not differentiate between all possible permutations of the different mark positions that are solutions to the problem, for example, 0-1-4-6, 4-6-1-0, 6-0-1-4, etc. As they are ORDER! (factorial of ORDER) such permutations, the search space would be drastically reduced by removing them.
<p>
We can do that by forcing an order between marks, for example the order of their index:

In [ ]:
# Avoid symmetric solutions by ordering marks
for i in range(1, ORDER):
    mdl.add(marks[i] > marks[i - 1])

We also know that first mark is at the beginning of the rule:

In [ ]:
# Force first mark position to zero
mdl.add(marks[0] == 0)

<h4>Avoid mirror solutions</h4>
<p>
Each optimal solution has a mirror, with all mark distances in the reverse order, for example, 01--4-6 and 0-2--56. 
The following constraint can be added to avoid this: 

In [ ]:
# Avoid mirror solution
mdl.add((marks[1] - marks[0]) < (marks[ORDER - 1] - marks[ORDER - 2]))

<h4>Define objective</h4>
<p>
Finally, we want to get the shortest Golomb Ruler. This can be expressed by minimizing the position of the last mark.
As we have ordered the marks, we can do this using:

In [ ]:
# Minimize ruler size
mdl.add(minimize(marks[ORDER - 1]))

If the marks were not ordered, we should have use instead:<br>
<code>   mdl.add(minimize(max(marks)))</code><br>

<h3>Solve the model</h3>
<p>
The model is now completely defined. It is time to solve it !
<p>
To use the CP Optimizer solver available on the IBM Decision Optimization on Cloud service:
<ul>
<li> Register for the DOcloud free trial and use it free for 30 days by using https://developer.ibm.com/docloud/try-docloud-free
<li> Get your access credentials (base URL and access key) by going this page: http://developer.ibm.com/docloud/docs/api-key/
</ul>
<p>


In [ ]:
# Initialize IBM Decision Optimization credentials
SVC_URL = "https://api-oaas.docloud.ibmcloud.com/job_manager/rest/v1/"
SVC_KEY = "ENTER YOUR KEY HERE"

The model can be solved by calling:

In [ ]:
# Solve the model
print("Solving model....")
msol = mdl.solve(url=SVC_URL, key=SVC_KEY)

<h4>Print the solution</h4>
<p>
The shortest way to output the solution that has been found by the solver is to call the method <i>print_solution()</i> as follows:

In [ ]:
# Print solution
print("Solution: ")
msol.print_solution()

This output is totally generic and simply prints the value of all model variables, the objective value, and some other solving information.
<p>
A more specific output can be generated by writing more code. The following example illustrates how to access specific elements of the solution. 

In [ ]:
# Print solution
from sys import stdout
if msol:
    # Print found solution
    stdout.write("Solution: " + msol.get_solve_status() + "\n")
    stdout.write("Position of ruler marks: ")
    for v in marks:
        stdout.write(" " + str(msol[v]))
    stdout.write("\n")
    stdout.write("Solve time: " + str(round(msol.get_solve_time(), 2)) + "s\n")
else:
    # No solution found
    stdout.write("No solution found. Search status: " + msol.get_solve_status() + "\n")

Another possibility is for example to simulate real ruler using characters, as follows:

In [ ]:
# Print solution as a ruler
if msol:
    stdout.write("Ruler: +")
    for i in range(1, ORDER):
        stdout.write('-' * (msol[marks[i]] - msol[marks[i - 1]] - 1) + '+')
    stdout.write("\n")

<h3>Going further with Constraint Programming</h3>
<p>
The last available installable package is available on Pypi here: https://pypi.python.org/pypi/docplex
<p>
Downloadable documentation is here: https://github.com/IBMDecisionOptimization/docplex-doc, and an inline version is available here:  http://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html
<p>
A complete set of modeling examples can be downloaded here: https://github.com/IBMDecisionOptimization/docplex-examples  